# Analisis de canasta de la compra

Definicion: El modelo de análisis de la canasta de la compra recorre los datos buscando cómo se relacionan entre ellos. Es decir, trata de encontrar las reglas de asociación que relacionan unos productos con otros en una compra.

Con la tabla que anteriormente realizamos usando R (canasta.md en github), podemos proceder con la ejecucion de este script de Python.

Al fin de la ejecucion, lograremos ver cuales productos tienen caracter complementarios, es decir, cuales venden con cual.

_Recordemos que para este analisis, dado al respeto de la privacidad los datos de mi cliente, y con su autorizacion,solo se uso una muestra pequeña de dos dias de trabajo, de solo una caja._ 

Comenzemos cargando las librerias necesarias, y cargando nuestra tabla previamente hecha. Guardaremos la tabla como 'Transacciones'

In [7]:
import pandas as pd
import numpy as np
from mlxtend.frequent_patterns import apriori
from mlxtend.frequent_patterns import association_rules

path_excel = r'C:\Users\Usuario\Desktop\Ventas\tickets\rojo\transacciones\\'

Transacciones = pd.read_excel(path_excel+'transacciones.xlsx')

Volvamos a ver la estructura de la tabla

In [9]:
Transacciones.head()

,Fecha,Hora,Transaccion,Codigo,Descripcion,Cant,Precio.Uni,Precio.Total,Articulos,Suma
0,2020-01-02,21:48:00,r1,414,CARAMELOS,1,2.00,2.0,2,30.0
1,2020-01-02,21:48:00,r1,0,JABON REXONA,1,28.00,28.0,2,30.0
2,2020-01-02,21:20:00,r10,100,QUESO FRESCO ECONOM,400,0.25,100.0,2,130.0
3,2020-01-02,21:20:00,r10,7791579001132,MOLTO PURE DE TOMAT,1,30.00,30.0,2,130.0
4,2020-01-02,13:22:00,r100,7790748235064,PUGLISI CABALLA EN,1,129.00,129.0,1,129.0


En nuestro estudio, no nos interesa saber que articulos venden junto a nuestros Productos Comunes (codigo de barra 0), entonces los eliminaremos.

In [10]:
tran_non0 = Transacciones[Transacciones['Codigo'] != 0]

El primer algoritmo que vamos a correr requiere que nuestra tabla este en forma de matriz. Como indice, quedara asignado el codigo unico de cada transacciones (numero de ticket), y las columnas seran todos los articulos comercializados en el negocio. Tomará una forma binaria en la cual, si en la columna del producto hay un 0, significa que para esa transaccion no se compro dicho producto, si hay un 1(luego pasaremos todos los valores mayor a 0 como un 1), significa que si se compro en esa transaccion. Por ende, modifiquemos la estructura de nuestra table:

In [11]:
tran_matrix = (tran_non0.groupby(['Transaccion','Codigo'])['Cant']
                .sum().unstack().reset_index().fillna(0)
                .set_index('Transaccion'))

In [12]:
tran_matrix.head()

Codigo,100,101,102,103,104,106,107,110,111,112,...,7798284590012,7798320512879,7798321150056,7798321150063,7798321150421,7798321150438,7798335287007,7891010010577,7891150053144,7891150053151
Transaccion,,,,,,,,,,,,,,,,,,,,,
r1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
r10,400.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
r100,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
r101,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
r102,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


Como nos interesa saber solo si el producto se compro, y no cuanto se compro, se convertira cualquier valor mayor a 0 a un 1. 

In [13]:
def encode_matrix(x):
    if x >= 1:
        return 1
    else:
        return 0
    
mi_canasta = tran_matrix.applymap(encode_matrix)

In [14]:
mi_canasta.head()

Codigo,100,101,102,103,104,106,107,110,111,112,...,7798284590012,7798320512879,7798321150056,7798321150063,7798321150421,7798321150438,7798335287007,7891010010577,7891150053144,7891150053151
Transaccion,,,,,,,,,,,,,,,,,,,,,
r1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
r10,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
r100,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
r101,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
r102,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [37]:
conjunto_art_frecuentes = apriori(mi_canasta, min_support=0.01, use_colnames=True)

Este algoritmo lo que calcula es la frecuencia en la cual aparecen cada articulo en la totalidad de las transacciones, de manera individual, o junto a otro articulo. Mientras mayor sea el porcentaje, con mayor frecuecia se vendio el/los articulo
(s). En el ejemplo abajo, los articulos numero 200 y 125 (que serian Pan y Salame) aparecen en ~1% de todas las transacciones, juntos.

Una vez ejecutado este algoritmo, se puede proceder con el algoritmo de reglas de asociacion.

In [64]:
conjunto_art_frecuentes

,support,itemsets
0,0.047138,(100)
1,0.050505,(101)
2,0.026936,(102)
3,0.026936,(103)
4,0.040404,(110)
...,...,...
99,0.010101,"(7798040058152, 110)"
100,0.010101,"(200, 125)"
101,0.010101,"(7790813110401, 7790199000020)"
102,0.010101,"(7795735000328, 7790270336307)"


In [39]:
reglas = association_rules(conjunto_art_frecuentes, metric="lift", min_threshold=1)

In [41]:
reglas

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction
0,(112),(101),0.026936,0.050505,0.010101,0.375000,7.425000,0.008741,1.519192
1,(101),(112),0.050505,0.026936,0.010101,0.200000,7.425000,0.008741,1.216330
2,(125),(102),0.020202,0.026936,0.013468,0.666667,24.750000,0.012924,2.919192
3,(102),(125),0.026936,0.020202,0.013468,0.500000,24.750000,0.012924,1.959596
4,(7798040058152),(110),0.057239,0.040404,0.010101,0.176471,4.367647,0.007788,1.165224
5,(110),(7798040058152),0.040404,0.057239,0.010101,0.250000,4.367647,0.007788,1.257015
6,(200),(125),0.043771,0.020202,0.010101,0.230769,11.423077,0.009217,1.273737
7,(125),(200),0.020202,0.043771,0.010101,0.500000,11.423077,0.009217,1.912458
8,(7790813110401),(7790199000020),0.030303,0.020202,0.010101,0.333333,16.500000,0.009489,1.469697
9,(7790199000020),(7790813110401),0.020202,0.030303,0.010101,0.500000,16.500000,0.009489,1.939394


Genial! En este resultado podemos apreciar cuales son los productos que se compran(consequents), dado a la compra de otro(antecedents). Las columnas a prestar mucha antencion son las de support, confidence y lift.

Support: Frecuencia en la cual aparece cada combinacion de productos, en relacion a todas las transacciones.

Confidence: Con qué frecuencia aparecen los articulos consecuentes en transacciones que contienen SOLO los articulos antecedentes, e.g, confidence dice que 37.5% de los clientes que compraron Jamon natural (codigo 112), tambien compraron Queso Fresco.

Lift: Cuánto ha aumentado nuestra confianza en que se comprará el producto consecuente dado que se compró el producto antecedente, e.g, el lift representa el aumento del 642.5% en la expectativa de que alguien compre queso fresco, cuando sabemos que compró jamon natural.

Para entender mejor, miremos dos casos con su Descripcion en vez de su codigo de barra:

In [142]:
print(Transacciones.loc[Transacciones['Codigo'] == 112,'Descripcion'].unique())
print(Transacciones.loc[Transacciones['Codigo'] == 101,'Descripcion'].unique())

['JAMON NATURAL ECONO']
['QUESO FRESCO PUNTA']


Con un lift y confianza alta, podemos observar que el jamon natural y el queso fresco son productos complementarios. Es probable que se compre jamon natural cuando se compre queso fresco, y MAS aun probable (confidence = 0.375) comprase queso fresco, cuando se compra antes el jamon natural. 

In [143]:
print(Transacciones.loc[Transacciones['Codigo'] == 7798040058152,'Descripcion'].unique())
print(Transacciones.loc[Transacciones['Codigo'] == 110,'Descripcion'].unique())

['EL COLOSO PAPEL HIG']
['PALETA JET FOOD']


Resultado curioso!, podemos observar con los datos disponibles,que el papel higienico El coloso y la paleta son productos complementarios.

In [144]:
print(Transacciones.loc[Transacciones['Codigo'] == 7790813110401,'Descripcion'].unique())
print(Transacciones.loc[Transacciones['Codigo'] == 7790199000020,'Descripcion'].unique())

['CALSA LEVADURA FRES']
['MORIXE HARINA DE TR']


In [ ]:
Los productos levadura y harina comun morixe arrojaron una asociacion muy alta entre las dos, como es de esperar.

_Recordemos que para este analisis, dado al respeto de la privacidad los datos de mi cliente, y con su autorizacion,solo se uso una muestra pequeña de dos dias de trabajo, de solo una caja._ 